In [1]:
import pymongo
from pymongo.mongo_client import MongoClient


uri = ""

# Create a new client and connect to the server
client = MongoClient(uri)
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)
    
    


Pinged your deployment. You successfully connected to MongoDB!


In [4]:
team_id = '65c0eff74dc9f94d6eec80e4'
sprint = 0


retro = list(db['Retro'].aggregate([
    {'$match': {'team_id': team_id, 'sprint': sprint}},
    {'$project': {
        'posts': [
            {'content': '$c1.content', 'responses': '$c1.responses'},
            {'content': '$c2.content', 'responses': '$c2.responses'},
            {'content': '$c3.content', 'responses': '$c3.responses'},
            {'content': '$c4.content', 'responses': '$c4.responses'}
        ]
    }},
    
    {'$unwind': '$posts'},
    
    {'$project': {
        'content': '$posts.content',
        'responses': '$posts.responses'}},
    
    {'$unwind': '$responses'},
    {'$project': {
        '_id': 0,
        'content': '$content',
        'note': '$responses.note',
        'thumb_up': '$responses.thumb_up',
        'thumb_down': '$responses.thumb_down',
        'diff_likes': {'$subtract': ['$responses.thumb_up', '$responses.thumb_down']}
    }},
    {'$match': {'note': {'$exists': True}} },
    {'$sort': {'diff_likes': -1}},
    {'$limit': 3}
]))


answers = list(db['Test1'].aggregate([
    {'$match': {'team_id': team_id, 'sprint': sprint}},
    {'$unwind': '$days'},
    {'$project': {
        'posts': [
            {'content': '$days.post.question1.content', 'value': '$days.post.question1.value'},
            {'content': '$days.post.question2.content', 'value': '$days.post.question2.value'},
            {'content': '$days.post.question3.content', 'value': '$days.post.question3.value'},
            {'content': '$days.post.question4.content', 'value': '$days.post.question4.value'},
        ],
        'date': '$days.date',  # Agregar el campo days.date
        'answers': '$days.answers'
    }},
    {'$unwind': '$posts'},
    {'$group': {
        '_id': '$posts.content',
        'average_value': {'$avg': {'$avg': '$posts.value'}},
        'date': {'$first': '$date'},  # Conservar el primer valor de days.date para cada agrupación
        'answers': {'$first': '$answers'} 
    }},
    {'$addFields': {
        '_id': {'$arrayElemAt': ['$_id', 0]} # Seleccionar solo el primer elemento del array _id
    }},
    {'$sort': {'average_value': 1}}
]))

days = list(db['Test1'].aggregate([
    {'$match': {'team_id': team_id, 'sprint': sprint}},
    {'$unwind': '$days'},
    {'$project': {
        '_id': 0,
        'date': '$days.date',
        'daily_self_satisfaction': {"$divide": ["$days.daily_self_satisfaction", "$days.answers"]},
        'daily_team_collaboration': {"$divide": ["$days.daily_team_collaboration", "$days.answers"]},
        'daily_work_engagement': {"$divide": ["$days.daily_work_engagement", "$days.answers"]},
        'daily_workspace_wellbeing': {"$divide": ["$days.daily_workspace_wellbeing", "$days.answers"]},
        'daily_general_satisfaction': {
            '$divide': [
                {'$add': [
                    {"$divide": ["$days.daily_self_satisfaction", "$days.answers"]},
                    {"$divide": ["$days.daily_team_collaboration", "$days.answers"]},
                    {"$divide": ["$days.daily_work_engagement", "$days.answers"]},
                    {"$divide": ["$days.daily_workspace_wellbeing", "$days.answers"]}
                ]},
                4
            ]
        }
    }},
    {'$sort': {'date': 1}}
]))

trending = {'cooperación': ['Adaptación al equipo y buen compañerismo'], 'Tiempo y velocidad': ['Falta de tiempo para testear implementaciones'], 'Presión jerárquica': ['Presión de los líderes del equipo', 'Traumas por corregir errores y falta de recursos para pagar psicólogo'], 'comodidad': ['Comidas poco llamativas en la oficina', 'Espacios de trabajo no confortables', 'Los espacios de trabajo no son suficientemente confortables']}

analysis = ''' 
En el análisis de la semana del equipo de trabajo, se observa una tendencia a la baja en las evaluaciones de satisfacción personal, colaboración en equipo, compromiso laboral, bienestar en el lugar de trabajo y satisfacción general durante los primeros días. Destaca especialmente la evaluación del día 2024-01-27 donde la satisfacción personal cae drásticamente a solo 0.1 debido a comentarios negativos respecto a las relaciones interpersonales en el equipo.

Sin embargo, hacia finales de la semana (2024-01-31), las evaluaciones muestran una mejora significativa con números más altos en todas las métricas, especialmente en bienestar en el lugar de trabajo y satisfacción general. Esto coincide con comentarios positivos sobre trabajar en equipo y un buen ambiente laboral.

Se identifican varios problemas a lo largo de la semana, como falta de comunicación, ineficiencias en el trabajo por falta de cooperación entre compañeros e incluso conflictos interpersonales. Estas situaciones pueden afectar directamente tanto al rendimiento como al bienestar general de los empleados.

Una posible solución para mejorar estas situaciones podría ser fomentar espacios abiertos para discutir problemas y buscar soluciones conjuntas, así como promover actividades que refuercen la cohesión del equipo. Además, considerar las necesidades individuales como la petición de un aire acondicionado para mejorar el bienestar físico también puede contribuir al mejoramiento general del ambiente laboral.

En resumen, es importante monitorear constantemente las métricas y comentarios para identificar tempranamente posibles áreas problemáticas y tomar medidas correctivas para mantener un ambiente laboral saludable y productivo.
'''

recommendation = '''
Recomendaciones:

1. Mejorar la comunicación y la colaboración en el equipo: 
   - Fomentar espacios de interacción más frecuentes y efectivos para evitar malentendidos y lograr una mejor coordinación en las tareas.
   - Establecer reuniones periódicas para discutir objetivos, avances y posibles obstáculos que estén impidiendo un trabajo eficiente.

2. Implementar acciones para fortalecer el compromiso laboral:
   - Realizar actividades que promuevan la integración del equipo y refuercen su sentido de pertenencia.
   - Reconocer públicamente los esfuerzos individuales y colectivos para motivar a los miembros del equipo a seguir comprometidos con sus responsabilidades.

3. Atender las necesidades de bienestar en el lugar de trabajo:
   - Tomar medidas concretas ante situaciones que afecten el confort y la productividad de los empleados, como garantizar un ambiente adecuadamente climatizado durante días calurosos.
   - Considerar implementar mejoras físicas en el espacio de trabajo para promover la organización y facilitar la colaboración entre los miembros del equipo.

4. Priorizar el desarrollo profesional y la diversidad en el entorno laboral:
    - Brindar recursos y oportunidades para mejorar habilidades profesionales, incentivando así la participación activa en programas de aprendizaje continuo.
    - Promover valores inclusivos dentro del equipo, respetando la diversidad y fomentando un ambiente laboral donde todos se sientan valorados y aceptados.

Enfocarse en estas áreas clave ayudará a mejorar el desempeño general del equipo, fortaleciendo su cohesión, compromiso y satisfacción laboral.
'''


report = { "retro": retro, "answers": answers, "days": days, "trending": trending, "analysis": analysis, "recommendation": recommendation}

report

{'retro': [{'content': '¿Qué debemos mantener?',
   'note': 'La calidad del trabajo que ofrecemos',
   'thumb_up': 3,
   'thumb_down': 0,
   'diff_likes': 3},
  {'content': '¿Qué debemos hacer más?',
   'note': 'Comunicarnos más. Reunirnos más frecuentemente.',
   'thumb_up': 2,
   'thumb_down': 0,
   'diff_likes': 2},
  {'content': '¿Qué debemos hacer más?',
   'note': 'Arriesgarnos y resolver al momento.',
   'thumb_up': 2,
   'thumb_down': 0,
   'diff_likes': 2}],
 'answers': [{'_id': 'En una escala del 1 al 10, ¿Cuánto participas en actividades de desarrollo de equipo?',
   'average_value': 0.7999999999999999,
   'date': datetime.datetime(2024, 2, 6, 0, 0),
   'answers': 3},
  {'_id': 'En una escala del 1 al 10, ¿Cómo calificarías tu capacidad para adaptarte a cambios inesperados?',
   'average_value': 0.8,
   'date': datetime.datetime(2024, 2, 5, 0, 0),
   'answers': 4},
  {'_id': 'En una escala del 1 al 10, ¿Cómo evalúas tu capacidad para trabajar de manera eficiente bajo presión

In [2]:
import openai

openai.api_key = ''

In [12]:
team_id = '65c0eff74dc9f94d6eec80e4'

sprint = 8


days = list(db['Test1'].aggregate([
    {'$match': {'team_id': team_id}},
    {'$unwind': '$days'},
    {'$project': {
        '_id': 0,
        'date': '$days.date',
        'daily_self_satisfaction': {"$divide": ["$days.daily_self_satisfaction", "$days.answers"]},
        'daily_team_collaboration': {"$divide": ["$days.daily_team_collaboration", "$days.answers"]},
        'daily_work_engagement': {"$divide": ["$days.daily_work_engagement", "$days.answers"]},
        'daily_workspace_wellbeing': {"$divide": ["$days.daily_workspace_wellbeing", "$days.answers"]},
        'daily_general_satisfaction': {
            '$divide': [
                {'$add': [
                    {"$divide": ["$days.daily_self_satisfaction", "$days.answers"]},
                    {"$divide": ["$days.daily_team_collaboration", "$days.answers"]},
                    {"$divide": ["$days.daily_work_engagement", "$days.answers"]},
                    {"$divide": ["$days.daily_workspace_wellbeing", "$days.answers"]}
                ]},
                4
            ]
        },
        'comments': {
            '$filter': {
                'input': {'$map': {
                    'input': '$days.post',
                    'as': 'post',
                    'in': '$$post.comment.content'
                }},
                'as': 'comment',
                'cond': {'$ne': ['$$comment', '']}
            }
        }
    }},
    {'$sort': {'date': 1}}
]))

retro = list(db['Retro'].aggregate([
    {'$match': {'team_id': team_id, 'sprint': sprint}},
    {'$project': {
        'posts': [
            {'content': '$c1.content', 'responses': '$c1.responses'},
            {'content': '$c2.content', 'responses': '$c2.responses'},
            {'content': '$c3.content', 'responses': '$c3.responses'},
            {'content': '$c4.content', 'responses': '$c4.responses'}
        ]
    }},
    
    {'$unwind': '$posts'},
    
    {'$project': {
        'content': '$posts.content',
        'responses': '$posts.responses'}},
    
    {'$unwind': '$responses'},
    {'$project': {
        '_id': 0,
        'content': '$content',
        'note': '$responses.note',
        'thumb_up': '$responses.thumb_up',
        'thumb_down': '$responses.thumb_down',
        'diff_likes': {'$subtract': ['$responses.thumb_up', '$responses.thumb_down']}
    }},
    {'$match': {'note': {'$exists': True}} },
    {'$sort': {'diff_likes': -1}},
    {'$limit': 3}
]))

answers = list(db['Test1'].aggregate([
    {'$match': {'team_id': team_id, 'sprint': sprint}},
    {'$unwind': '$days'},
    {'$project': {
        'posts': [
            {'content': '$days.post.question1.content', 'value': '$days.post.question1.value'},
            {'content': '$days.post.question2.content', 'value': '$days.post.question2.value'},
            {'content': '$days.post.question3.content', 'value': '$days.post.question3.value'},
            {'content': '$days.post.question4.content', 'value': '$days.post.question4.value'},
        ],
        'date': '$days.date',  # Agregar el campo days.date
        'answers': '$days.answers'
    }},
    {'$unwind': '$posts'},
    {'$group': {
        '_id': '$posts.content',
        'average_value': {'$avg': {'$avg': '$posts.value'}},
        'date': {'$first': '$date'},  # Conservar el primer valor de days.date para cada agrupación
        'answers': {'$first': '$answers'} 
    }},
    {'$addFields': {
        '_id': {'$arrayElemAt': ['$_id', 0]} # Seleccionar solo el primer elemento del array _id
    }},
    {'$sort': {'average_value': 1}},
    {'$limit': 3}
]))

prompt = '''
Te daré una serie de métricas recolectadas de un equipo de trabajo de IT. 
Tras analizar detenidamente y en profundidad los datos, generarás recomendaciones en base a los problemas que encuentres o cambios bruscos que puedan indicar una problemática.
Usarás las métricas que caracterizan la semana y los comentarios de las personas a lo largo de los días para brindar recomendaciones a este equipo de trabajo.
Las recomendaciones deben aportar valor a este equipo de trabajo sobre posibles soluciones a los problemas más importante que hayas encontrado. 

DATOS:

- Puntuaciónes a lo largo de la semana y comentarios.
'''

for day in days:
    prompt += f'''
Fecha: {day['date'].date()}
Evaluación general de la satisfacción personal: {round(day['daily_self_satisfaction'], 2)}
Evaluación de colaboración en equipo: {round(day['daily_team_collaboration'], 2)}
Evaluación de compromiso laboral: {round(day['daily_work_engagement'], 2)}
Evaluación de bienestar en el lugar de trabajo: {round(day['daily_workspace_wellbeing'], 2)}
Evaluación de la satisfacción general: {round(day['daily_general_satisfaction'], 2)}

Comentarios del día:'''

    if day['comments']:
        for comment in day['comments']:
            prompt += f"\n\t- {comment}"
    else:
        prompt += '\n\t- No hubo comentarios'

prompt += '''
- Respuestas más gustadas y populares de la Retrospectiva
'''

for notes in retro:
    prompt += f'''
Pregunta de la retrospectiva: {notes['content']}
Respuesta: {notes['note']}
'''


prompt += '''Devolverás únicamente las recomendaciones sin usar negrita pudiendo enumerar y usar guiones de ser necesario. Es preferible tener pocas recomendaciones pero bien desarrolladas que demasiadas mal hechas'''

prompt += '''
- Preguntas con peor puntuación en el sprint.
'''

for answer in answers:
    prompt += f'''
Pregunta de la retrospectiva: {answer['_id']}
Puntaje: { round(answer['average_value'], 2) }
'''


completion = openai.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
    {
        "role": "system",
        "content": "Eres un líder de proyecto experto y debes ayudar a este equipo de trabajo",
    },
    {
        "role": "user",
        "content": prompt,
    },
],
    temperature= 1.1,
    presence_penalty = 0.75,
    frequency_penalty = 0.5
    
)
#print(prompt)
print(completion.choices[0].message.content)


Recomendaciones:

1. Mejorar la comunicación y la colaboración en el equipo: 
   - Fomentar espacios de interacción más frecuentes y efectivos para evitar malentendidos y lograr una mejor coordinación en las tareas.
   - Establecer reuniones periódicas para discutir objetivos, avances y posibles obstáculos que estén impidiendo un trabajo eficiente.

2. Implementar acciones para fortalecer el compromiso laboral:
   - Realizar actividades que promuevan la integración del equipo y refuercen su sentido de pertenencia.
   - Reconocer públicamente los esfuerzos individuales y colectivos para motivar a los miembros del equipo a seguir comprometidos con sus responsabilidades.

3. Atender las necesidades de bienestar en el lugar de trabajo:
   - Tomar medidas concretas ante situaciones que afecten el confort y la productividad de los empleados, como garantizar un ambiente adecuadamente climatizado durante días calurosos.
   - Considerar implementar mejoras físicas en el espacio de trabajo para 

In [ ]:
prompt += '''
- Preguntas con peor puntuación en el sprint.
'''

for answer in answers:
    prompt += f'''
Pregunta de la retrospectiva: {answer['_id']}
Puntaje: { round(answer['average_value'], 2) }
'''